# About this Jupyter Notebook

@author: Yingding Wang

From the discussion of minio error on pipeline ui:
https://github.com/kubeflow/pipelines/issues/3819

In [1]:
import sys

In [2]:
!{sys.executable} -m pip install --upgrade --user kfp==1.8.12

In [3]:
!{sys.executable} -m pip list | grep kfp

kfp                          1.8.12
kfp-pipeline-spec            0.1.14
kfp-server-api               1.7.0


In [4]:
from kfp.dsl import ContainerOp
def pod_resource_transformer(op: ContainerOp, mem_req="200Mi", cpu_req="2000m", mem_lim="1000Mi", cpu_lim='2000m'):
    """
    op.set_memory_limit('1000Mi') = 1GB
    op.set_cpu_limit('1000m') = 1 cpu core
    """
    return op.set_memory_request(mem_req)\
            .set_memory_limit(mem_lim)\
            .set_cpu_request(cpu_req)\
            .set_cpu_limit(cpu_lim)

In [5]:
from typing import NamedTuple
import kfp
from kfp.components import create_component_from_func
from kfp.dsl import pipeline

@create_component_from_func
def produce_markdown() -> NamedTuple('Outputs', [('MLPipeline_UI_metadata', 'UI_metadata')]):
    import json
    markdown = {"outputs": [{"storage": "inline", "source": "# TEST AUC:xxxxxx \n\n#GBDT AUC:xxxxxxx", "type": "markdown"}]}
    return [json.dumps(markdown)]    

@pipeline(name="plot_pipeline")
def plot_pipeline():
    plot_task = produce_markdown()
    plot_task = pod_resource_transformer(plot_task, mem_req="500Mi")
    plot_task.execution_options.caching_strategy.max_cache_staleness = "P0D"


custom_pipeline = plot_pipeline

In [6]:
PIPE_LINE_FILE_NAME="kfp_14_markdown_plot_pipeline"
kfp.compiler.Compiler().compile(custom_pipeline, f"{PIPE_LINE_FILE_NAME}.yaml")

### Run Pipeline

In [7]:
NAME_SPACE="kubeflow-kindfor"
EXPERIMENT_NAME="kf-metrics"

In [8]:
kfp_endpoint=None
client = kfp.Client(host=kfp_endpoint)
client.set_user_namespace(NAME_SPACE)

In [9]:
client.create_run_from_pipeline_func(pipeline_func=custom_pipeline, arguments={},
                                     experiment_name=EXPERIMENT_NAME, namespace=NAME_SPACE,
                                     pipeline_conf=kfp.dsl.PipelineConf(),
                                     mode = kfp.dsl.PipelineExecutionMode.V1_LEGACY
                                    )

RunPipelineResult(run_id=b9057c7d-9cdd-418e-b34c-be084a651c78)